# 1 prey 1 predator model (Wild Boar and Wolf)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy import signal, integrate, stats

In [2]:
# Wild boar and wolf
alpha2 = 6.22 # Birth rate prey 
beta2 = 12.4/182.75  # Death rate prey
gamma2 = 25/26 # Death rate predator
delta2 = 0.0356 # Grow rate predator

# Time * step = amount of years simulated
time2 = 1000000 
step2 = 0.0001

# Initial populations
init_wolf2 = 1
init_boar2 = 500

In [3]:
def wolf_eq(gamma, delta, w, z, step):
    """Differential equation for the wolf population."""
    return step * w * (-gamma + delta*z)


def boar_eq(alpha, beta, w, z, step):
    """Differential equation for the wild boar population."""
    return step * z * (alpha - beta*w)

In [4]:
def update_single_prey(init_wolf2, init_boar2, alpha2, beta2, gamma2, delta2, step2):
    """Plot lotka-volterra equations for 1 predator and 1 prey population."""
    # Initialize the initial populations.
    wolf2 = [init_wolf2]
    boar2 = [init_boar2]

    num_wolf2 = init_wolf2
    num_boar2 = init_boar2
    
    # Calculate the data points and store them in an array.
    for n in range(time2):
        dwolf2 = wolf_eq(gamma2, delta2, num_wolf2, num_boar2, step2)
        dboar2 = boar_eq(alpha2, beta2, num_wolf2, num_boar2, step2)
        num_wolf2 += dwolf2
        num_boar2 += dboar2
        wolf2.append(num_wolf2)
        boar2.append(num_boar2)
        
    
    # Make the time array to set as the x-values.
    t2 = range(time2+1)
    t2 = [step2*x for x in t2]
    
    # Plot the figure.
    plt.figure(figsize=(12,8))
    plt.plot(t2, wolf2)
    plt.plot(t2, boar2)
    plt.ylabel('Number of animals')
    plt.xlabel('Time (years)')
    plt.title('Wolf-Wild boar model', fontsize=20)
    plt.legend(['Wolf', 'Wild boar'], loc='upper left')
    plt.show()

# Create sliders for all variables so that the plot can be updated easily.
iwolf2_slider = widgets.IntSlider(value=init_wolf2, min=0, max=100, step=1)
iboar2_slider = widgets.IntSlider(value=init_boar2, min=0, max=300, step=1)
a2_slider = widgets.FloatSlider(value=alpha2, min=0, max=1, step=0.01)
b2_slider = widgets.FloatSlider(value=beta2, min=0, max=1, step=0.01)
g2_slider = widgets.FloatSlider(value=gamma2, min=0, max=1, step=0.01)
d2_slider = widgets.FloatSlider(value=delta2, min=0, max=1, step=0.01)
s2_slider = widgets.FloatSlider(value=step2, min=0, max=10, step=0.001)

# Draw the sliders and set them to the correct variable.
widgets.interact_manual(update_single_prey, init_wolf2=iwolf2_slider, init_boar2=iboar2_slider, alpha2 = a2_slider,
                        beta2 = b2_slider, gamma2 = g2_slider, delta2 = d2_slider, step2=s2_slider);

interactive(children=(IntSlider(value=1, description='init_wolf2'), IntSlider(value=300, description='init_boa…

# Lotka-Volterra Vector Field (1 prey 1 predator) 

In [5]:
alpha3 = 9/11 # Birth rate prey 
beta3 = 48.6/365.25 # Death rate prey
gamma3 = 25/26 # Death rate predator
delta3 = 0.0356 # Grow rate predator

max_prey = 50 # Maximum begin population of prey
max_pred = 15 # Maximum begin population of predators

arrow_density = 2 # Number of arrows per 1 unit

# Make a sample starting point.
sample_pred = 7
sample_prey = 29
sample_length = 400
sample_step = 0.1
arrows = True
arrow_distance = 10

In [6]:
def update_vector(alpha3, beta3, gamma3, delta3, max_prey, max_pred, arrow_density,
                  sample_pred, sample_prey, sample_length, sample_step, arrows, arrow_distance):
    """Plots a vector field for different initial populations of 1 predator and 1 prey."""
    
    # Calculate the stable initial populations for the given coefficients.
    stable_pred = alpha3 / beta3
    stable_prey = gamma3 / delta3
    print('The stable point (besides (0,0)) is at %.2f initial preys and %.2f initial predators.' 
          % (stable_prey, stable_pred))

    # Check stability in the differential equations.
    print('Predator difference (should be 0): %.2f' % wolf_eq(gamma3, delta3, stable_pred, stable_prey, 1))
    print('Prey difference (should be 0): %.2f' % boar_eq(alpha3, beta3, stable_pred, stable_prey, 1))
    
    # Initialize the field and the arrays for change in predator and prey populations.
    X, Y = np.meshgrid(np.arange(0, max_prey, 1/arrow_density), np.arange(0, max_pred, 1/arrow_density))
    U = np.zeros(X.shape)
    V = np.zeros(X.shape)
    
    # Make the data for the vector directions.
    for r in range(max_prey * arrow_density):
        for c in range(max_pred * arrow_density):
            U[c][r] = boar_eq(alpha3, beta3, c/arrow_density, r/arrow_density, 1) 
            V[c][r] = wolf_eq(gamma3, delta3, c/arrow_density, r/arrow_density, 1) 
            norm = np.sqrt((U[c][r])**2 + (V[c][r])**2)
            if norm != 0:
                U[c][r] = U[c][r] / norm
                V[c][r] = V[c][r] / norm
    
    # Calculate how the sample initial point changes with time. 
    s_prey = [sample_prey]
    s_pred = [sample_pred]
    for t in range(sample_length):
        cur_prey = s_prey[-1]
        cur_pred = s_pred[-1]
        s_prey.append(cur_prey + boar_eq(alpha3, beta3, cur_pred, cur_prey, sample_step))
        s_pred.append(cur_pred + wolf_eq(gamma3, delta3, cur_pred, cur_prey, sample_step))
    
    # Plot the vectors with the quiver function.
    plt.figure(figsize=(15,14))
    plt.quiver(X, Y, U, V, scale=90, width=0.0015, color='b') # bigger scale is smaller arrows

    # Draw the sample line.
    plt.plot(s_prey, s_pred, 'r-')

    # Draw starting point as a dot.
    plt.plot(sample_prey, sample_pred, 'ro')

    # Draw arrows in line. 
    if arrows:
        for i in range(9, sample_length, arrow_distance):
            if i+1 < sample_length:
                plt.arrow(s_prey[i], s_pred[i], s_prey[i+1]-s_prey[i], s_pred[i+1]-s_pred[i], color='r', head_width=0.2)
    
    # Draw information on the plot and show it.
    plt.legend(['Example start point (%d prey, %d predators)' % (sample_prey, sample_pred)])
    plt.ylabel('Predator population')
    plt.xlabel('Prey population')
    plt.title('Vector Field of Lotka-Volterra equation', fontsize=20)
    plt.show()


# Make all the sliders for the different variables so that the plot can be updated easily.
a3_slider = widgets.FloatSlider(value=alpha3, min=0, max=1, step=0.01)
b3_slider = widgets.FloatSlider(value=beta3, min=0, max=1, step=0.01)
g3_slider = widgets.FloatSlider(value=gamma3, min=0, max=1, step=0.01)
d3_slider = widgets.FloatSlider(value=delta3, min=0, max=1, step=0.01)
maxpred_slider = widgets.IntSlider(value=max_pred, min=0, max=100, step=1)
maxprey_slider = widgets.IntSlider(value=max_prey, min=0, max=300, step=1)

arrowdense_slider = widgets.IntSlider(value=arrow_density, min=0, max=10, step=1)

sampred_slider = widgets.IntSlider(value=sample_pred, min=0, max=100, step=1)
samprey_slider = widgets.IntSlider(value=sample_prey, min=0, max=100, step=1)
samplen_slider = widgets.IntSlider(value=sample_length, min=0, max=1000, step=10)

sampstep_slider = widgets.FloatSlider(value=sample_step, min=0, max=10, step=0.001)
arrowdist_slider = widgets.IntSlider(value=arrow_distance, min=0, max=100, step=1)

# Draw the sliders and set them to the correct variable.
widgets.interact_manual(update_vector, alpha3=a3_slider, beta3=b3_slider, gamma3=g3_slider, delta3=d3_slider, 
                max_prey=maxprey_slider, max_pred=maxpred_slider, arrow_density=arrowdense_slider,
                sample_pred=sampred_slider, sample_prey=samprey_slider, sample_length=samplen_slider, 
                sample_step=sampstep_slider, arrows=True, arrow_distance=arrowdist_slider);

interactive(children=(FloatSlider(value=0.8181818181818182, description='alpha3', max=1.0, step=0.01), FloatSl…